In [1]:
from google.colab import drive
try:
    drive.flush_and_unmount()
except Exception:
    pass
import shutil, os, time
shutil.rmtree("/content/drive", ignore_errors=True)
time.sleep(1)
drive.mount("/content/drive", force_remount=True)


Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/code_data25/Basu

/content/drive/MyDrive/code_data25/Basu


In [9]:
# ===== MLP with per-class 80/20 split (train/test), CFO/RSS skipped =====
!pip -q install h5py scikit-learn

import h5py, numpy as np, torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
H5_PATH = "Dataset-RFFI/dataset_training_no_aug.h5"   # <-- set your path
BATCH = 512
EPOCHS = 1000
LR = 1e-3
HIDDEN = 512
STANDARDIZE = True   # per-feature mean/std on /data
RNG_SEED = 42        # reproducible split

# -------- Dataset: only /data and /label --------
class WifiH5NoMeta(Dataset):
    def __init__(self, path, standardize=True):
        self.f = h5py.File(path, "r")
        self.X = self.f["/data"]                         # (N, D) float64
        y_raw = np.asarray(self.f["/label"][0], np.int64)
        uniq = np.unique(y_raw); self.label_map = {int(v): i for i,v in enumerate(uniq)}
        self.y = np.array([self.label_map[int(v)] for v in y_raw], np.int64)

        self.mu = self.sd = None
        if standardize:
            # chunked mean/std so we don't load all into RAM
            n, d = self.X.shape
            m = np.zeros(d, np.float64); v = np.zeros(d, np.float64); cnt = 0
            bs = 512
            for i in range(0, n, bs):
                chunk = self.X[i:i+bs].astype(np.float64)
                m_c, v_c = chunk.mean(axis=0), chunk.var(axis=0)
                if cnt == 0:
                    m[:] = m_c; v[:] = v_c * chunk.shape[0]; cnt = chunk.shape[0]
                else:
                    n2 = cnt + chunk.shape[0]; delta = m_c - m
                    m += delta * (chunk.shape[0] / n2)
                    v += v_c * chunk.shape[0] + (delta**2) * (cnt * chunk.shape[0] / n2)
                    cnt = n2
            var = v / max(cnt-1,1)
            self.mu = torch.from_numpy(m.astype(np.float32))
            self.sd = torch.from_numpy(np.sqrt(var + 1e-8).astype(np.float32))

    def __len__(self): return self.X.shape[0]

    def __getitem__(self, idx):
        x = torch.from_numpy(self.X[idx].astype(np.float32))   # (D,)
        if self.mu is not None: x = (x - self.mu) / self.sd
        y = torch.tensor(self.y[idx], dtype=torch.long)
        return x, y

# -------- Build per-class 80/20 split (train/test) --------
full = WifiH5NoMeta(H5_PATH, standardize=STANDARDIZE)
N, D = full.X.shape
num_classes = len(full.label_map)
print(f"N={N}, D={D}, classes={num_classes}")
print("label_map:", full.label_map)

labels = np.array(full.y)
rng = np.random.default_rng(RNG_SEED)

train_idx_list, test_idx_list = [], []
for c in np.unique(labels):
    idx = np.where(labels == c)[0]
    rng.shuffle(idx)
    n_c = len(idx)
    n_train = int(np.floor(0.8 * n_c))
    # ensure both splits non-empty when possible
    if n_train == 0 and n_c >= 2: n_train = 1
    if n_c - n_train == 0 and n_c >= 2: n_train = n_c - 1
    train_idx_list.append(idx[:n_train])
    test_idx_list.append(idx[n_train:])

train_idx = np.concatenate(train_idx_list)
test_idx  = np.concatenate(test_idx_list)

# Show per-class counts for sanity
print("\nPer-class counts (train/test):")
for c in np.unique(labels):
    tr = np.sum(labels[train_idx] == c)
    te = np.sum(labels[test_idx]  == c)
    print(f"class {c}: train={tr}, test={te}, total={tr+te}")

from torch.utils.data import DataLoader, Subset
train_ds, test_ds = Subset(full, train_idx), Subset(full, test_idx)
train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True,  num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True)

# -------- Simple MLP: Linear -> ReLU -> Linear --------
class TinyMLP(nn.Module):
    def __init__(self, in_dim, num_classes, hidden=HIDDEN):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, hidden)
        self.fc2 = nn.Linear(hidden, num_classes)
        self.act = nn.ReLU()
    def forward(self, x):
        return self.fc2(self.act(self.fc1(x)))

model = TinyMLP(D, num_classes).to(DEVICE)

cls, cnt = np.unique(labels[train_idx], return_counts=True)
weights = torch.tensor(cnt.mean() / cnt, dtype=torch.float32, device=DEVICE)
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)


# -------- Train / Test --------
def run_epoch(loader, train=True):
    (model.train() if train else model.eval())
    total, correct, loss_sum = 0, 0, 0.0
    with torch.set_grad_enabled(train):
        for xb, yb in loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            if train: optimizer.zero_grad(set_to_none=True)
            logits = model(xb)
            loss = criterion(logits, yb)
            if train:
                loss.backward()
                optimizer.step()
            loss_sum += loss.item() * yb.size(0)
            correct += (logits.argmax(1) == yb).sum().item()
            total += yb.size(0)
    return loss_sum/total, correct/max(1,total)


best_acc = 0.0
for ep in range(1, EPOCHS+1):
    tr_loss, tr_acc = run_epoch(train_loader, True)
    te_loss, te_acc = run_epoch(test_loader,  False)
    print(f"Epoch {ep:02d} | train {tr_loss:.4f}/{tr_acc:.4f} | test {te_loss:.4f}/{te_acc:.4f}")
    if te_acc > best_acc:
        best_acc = te_acc
        torch.save({
            "model": model.state_dict(),
            "label_map": full.label_map,
            "in_dim": D,
            "num_classes": num_classes
        }, "/content/drive/MyDrive/code_data25/Basu/model_save/best_router_mlp_80_20.pth")

# -------- Final report (on test set) --------
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        pr = model(xb.to(DEVICE)).argmax(1).cpu().numpy()
        y_pred.append(pr); y_true.append(yb.numpy())
y_true = np.concatenate(y_true); y_pred = np.concatenate(y_pred)

print("\nTest accuracy:", accuracy_score(y_true, y_pred))
print("\nPer-class report:\n", classification_report(y_true, y_pred, digits=4))
print("\nConfusion matrix (rows=true, cols=pred):\n", confusion_matrix(y_true, y_pred))


N=15000, D=16384, classes=30
label_map: {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29}

Per-class counts (train/test):
class 0: train=400, test=100, total=500
class 1: train=400, test=100, total=500
class 2: train=400, test=100, total=500
class 3: train=400, test=100, total=500
class 4: train=400, test=100, total=500
class 5: train=400, test=100, total=500
class 6: train=400, test=100, total=500
class 7: train=400, test=100, total=500
class 8: train=400, test=100, total=500
class 9: train=400, test=100, total=500
class 10: train=400, test=100, total=500
class 11: train=400, test=100, total=500
class 12: train=400, test=100, total=500
class 13: train=400, test=100, total=500
class 14: train=400, test=100, total=500
class 15: train=400, test=100, total=500
class 16: train=400, test=100, total=500
class 17: train=400

In [11]:
import numpy as np, json
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import torch

# === load your trained model weights ===
CKPT_PATH = "/content/drive/MyDrive/code_data25/Basu/model_save/best_router_mlp_80_20.pth"  # <-- change if needed
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# If you saved a dict with {"model": state_dict, ...}
ckpt = torch.load(CKPT_PATH, map_location=DEVICE)
state_dict = ckpt["model"] if isinstance(ckpt, dict) and "model" in ckpt else ckpt

model.to(DEVICE)
missing, unexpected = model.load_state_dict(state_dict, strict=False)
print("Loaded weights. Missing keys:", missing, "| Unexpected keys:", unexpected)

# write label_map if present
if isinstance(ckpt, dict) and "label_map" in ckpt:
    with open("label_map.json", "w") as f:
        json.dump(ckpt["label_map"], f, indent=2)

# === evaluation + saving ===
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        pr = model(xb.to(DEVICE)).argmax(1).cpu().numpy()
        y_pred.append(pr); y_true.append(yb.numpy())
y_true = np.concatenate(y_true); y_pred = np.concatenate(y_pred)

acc = accuracy_score(y_true, y_pred)
rep = classification_report(y_true, y_pred, digits=4)
cm  = confusion_matrix(y_true, y_pred)

with open("/content/drive/MyDrive/code_data25/Basu/save_results/test_accuracy.txt", "w") as f:
    f.write(f"{acc:.6f}\n")

with open("/content/drive/MyDrive/code_data25/Basu/save_results/classification_report.txt", "w") as f:
    f.write(rep + "\n")

np.savetxt("/content/drive/MyDrive/code_data25/Basu/save_results/confusion_matrix.txt", cm, fmt="%d")

np.savetxt(
    "/content/drive/MyDrive/code_data25/Basu/save_results/predictions.csv",
    np.column_stack([y_true, y_pred]),
    fmt="%d",
    delimiter=",",
    header="y_true,y_pred",
    comments=""
)

df = pd.DataFrame({"y_true": y_true, "y_pred": y_pred})
per_class_acc = df.groupby("y_true").apply(lambda g: float((g.y_true == g.y_pred).mean()))
per_class_acc.to_csv("/content/drive/MyDrive/code_data25/Basu/save_results/per_class_accuracy.csv", header=["accuracy"])

print("Saved: test_accuracy.txt, classification_report.txt, confusion_matrix.txt, predictions.csv, per_class_accuracy.csv")

# ========= single-sample latency + GFLOPs report =========
import os, time, json
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn

OUT_DIR = "/content/drive/MyDrive/code_data25/Basu/save_results"
os.makedirs(OUT_DIR, exist_ok=True)

# 1) Take one real sample from your test loader
model.eval()
xb_one, _ = next(iter(test_loader))          # uses your real preprocessing pipeline
xb_one = xb_one[:1].to(DEVICE)               # shape [1, D]

# 2) Accurate timing (GPU-aware)
def time_inference(model, x, warmup=30, repeats=200):
    with torch.no_grad():
        # warmup (not measured)
        for _ in range(warmup):
            _ = model(x)
            if x.is_cuda:
                torch.cuda.synchronize()

        times = []
        if x.is_cuda:
            start = torch.cuda.Event(enable_timing=True)
            end   = torch.cuda.Event(enable_timing=True)
            for _ in range(repeats):
                start.record(); _ = model(x); end.record()
                torch.cuda.synchronize()
                times.append(start.elapsed_time(end) / 1000.0)  # seconds
        else:
            for _ in range(repeats):
                t0 = time.perf_counter(); _ = model(x); t1 = time.perf_counter()
                times.append(t1 - t0)

    times = np.array(times, dtype=np.float64)
    return {
        "device": "cuda" if x.is_cuda else "cpu",
        "batch_size": int(x.shape[0]),
        "runs": int(repeats),
        "mean_s": float(times.mean()),
        "median_s": float(np.median(times)),
        "p90_s": float(np.percentile(times, 90)),
        "p95_s": float(np.percentile(times, 95)),
        "min_s": float(times.min()),
        "max_s": float(times.max()),
    }

# 3) FLOPs for MLP (sum over Linear layers): 1 multiply + 1 add = 2 FLOPs
def mlp_flops_per_sample(model):
    flops = 0
    for m in model.modules():
        if isinstance(m, nn.Linear):
            flops += 2 * m.in_features * m.out_features
    return flops  # per-sample

def num_params(model):
    return sum(p.numel() for p in model.parameters())

timing = time_inference(model, xb_one, warmup=30, repeats=200)
flops  = mlp_flops_per_sample(model)
report = {
    "device": timing["device"],
    "batch_size": timing["batch_size"],
    "runs": timing["runs"],
    "per_sample_time_s_mean": timing["mean_s"],
    "per_sample_time_s_median": timing["median_s"],
    "per_sample_time_s_p90": timing["p90_s"],
    "per_sample_time_s_p95": timing["p95_s"],
    "per_sample_time_s_min": timing["min_s"],
    "throughput_samples_per_s_mean": (1.0 / timing["mean_s"]) if timing["mean_s"] > 0 else None,
    "params_total": int(num_params(model)),
    "flops_per_sample": int(flops),
    "gflops_per_sample": float(flops / 1e9),
}

print(json.dumps(report, indent=2))
Path(f"{OUT_DIR}/inference_profile.json").write_text(json.dumps(report, indent=2))
print(f"Saved {OUT_DIR}/inference_profile.json")

# ========= per-class bar, ROC curves, and t-SNE (save PNGs) =========
import os, json
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.manifold import TSNE
import pandas as pd

OUT_DIR = "/content/drive/MyDrive/code_data25/Basu/save_results"
os.makedirs(OUT_DIR, exist_ok=True)

# ---------- A) Per-class accuracy bar plot ----------
df_eval = pd.DataFrame({"y_true": y_true, "y_pred": y_pred})
per_class_acc = df_eval.groupby("y_true").apply(lambda g: float((g.y_true == g.y_pred).mean()))
per_class_acc = per_class_acc.sort_values(ascending=True)  # ascending so worst classes at top

plt.figure(figsize=(7, max(4, 0.22 * len(per_class_acc))), dpi=400)
plt.barh(per_class_acc.index.astype(str), per_class_acc.values)
plt.xlabel("Per-class Accuracy")
plt.ylabel("Class ID")
plt.title("Per-class Accuracy (Sorted)")
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/per_class_accuracy_bar.png", bbox_inches="tight", facecolor="white")
plt.close()

# ---------- B) ROC curves (micro/macro + OvR) ----------
# We need class probabilities.
model.eval()
all_scores, all_labels = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        logits = model(xb.to(DEVICE))
        probs = torch.softmax(logits, dim=1).cpu().numpy()
        all_scores.append(probs)
        all_labels.append(yb.numpy())
y_score = np.vstack(all_scores)  # shape [N, C]
y_true_arr = np.concatenate(all_labels)  # shape [N]
classes = np.unique(y_true_arr)
n_classes = classes.size

# Binarize labels for multi-class ROC
y_true_bin = label_binarize(y_true_arr, classes=classes)  # shape [N, C]

# Compute micro, macro, and OvR ROC
fpr, tpr, roc_auc = {}, {}, {}
# OvR for each class (columns aligned with 'classes')
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# micro-average
fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# macro-average
# compute average of the interpolated TPRs
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i], left=0, right=1)
mean_tpr /= n_classes
fpr["macro"], tpr["macro"] = all_fpr, mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot
fig = plt.figure(figsize=(7, 6), dpi=400, facecolor="white")
ax = fig.add_subplot(111)
ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1)

# Emphasize micro & macro
ax.plot(fpr["micro"], tpr["micro"], linewidth=2, label=f"micro-average (AUC = {roc_auc['micro']:.3f})")
ax.plot(fpr["macro"], tpr["macro"], linewidth=2, label=f"macro-average (AUC = {roc_auc['macro']:.3f})")

# Plot a few one-vs-rest class curves (to avoid clutter if many classes)
MAX_OVR = 8
pick = np.linspace(0, n_classes-1, min(n_classes, MAX_OVR), dtype=int)
for i in pick:
    ax.plot(fpr[i], tpr[i], linewidth=1, label=f"class {classes[i]} (AUC = {roc_auc[i]:.3f})")

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curves: micro, macro, and One-vs-Rest")
ax.legend(loc="lower right", fontsize=7)
fig.tight_layout()
fig.savefig(f"{OUT_DIR}/roc_curves_micro_macro_onevsrest.png", bbox_inches="tight", facecolor="white")
plt.close(fig)

# ---------- C) t-SNE on penultimate features (fallback to logits if needed) ----------
# We’ll try to grab the penultimate activations by registering a forward hook on the last Linear layer’s input.
# If that fails, we’ll use logits as features.
last_linear = None
for m in model.modules():
    if isinstance(m, nn.Linear):
        last_linear = m
# Container to hold features from the forward hook
_feats = []
def hook_fn(module, inputs, output):
    # inputs is a tuple; we want the input activations to this linear layer (penultimate features)
    _feats.append(inputs[0].detach().cpu())

h = None
if last_linear is not None:
    h = last_linear.register_forward_hook(hook_fn)

# Run forward passes to collect features
model.eval()
feat_list, lab_list = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        _ = model(xb.to(DEVICE))
        if _feats:
            feat_list.append(_feats[-1])  # latest batch’s features
        else:
            # Fallback to logits if hook didn't fire
            feat_list.append(_.detach().cpu())
        lab_list.append(yb)

# Cleanup hook
if h is not None:
    h.remove()

feats = torch.cat(feat_list, dim=0).cpu().numpy()
labs  = torch.cat(lab_list, dim=0).cpu().numpy()

# To keep t-SNE stable and fast, optionally subsample if huge
MAX_TSNE = 5000
if feats.shape[0] > MAX_TSNE:
    rng = np.random.default_rng(42)
    idx = rng.choice(feats.shape[0], size=MAX_TSNE, replace=False)
    feats_vis = feats[idx]
    labs_vis  = labs[idx]
else:
    feats_vis, labs_vis = feats, labs

tsne = TSNE(n_components=2, perplexity=30, learning_rate="auto", init="pca", random_state=42)
emb  = tsne.fit_transform(feats_vis)

plt.figure(figsize=(7, 6), dpi=400)
sc = plt.scatter(emb[:,0], emb[:,1], c=labs_vis, s=5, alpha=0.8)
plt.title("t-SNE of Learned Representations (penultimate features)")
plt.xlabel("t-SNE dim 1"); plt.ylabel("t-SNE dim 2")
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/tsne_features.png", bbox_inches="tight", facecolor="white")
plt.close()

print("Saved:",
      f"{OUT_DIR}/per_class_accuracy_bar.png,",
      f"{OUT_DIR}/roc_curves_micro_macro_onevsrest.png,",
      f"{OUT_DIR}/tsne_features.png")
# ========= END ADD =========



import json, os
import numpy as np
import matplotlib.pyplot as plt

# class names from label_map
class_names = None
if os.path.exists("label_map.json"):
    with open("label_map.json") as f:
        lm = json.load(f)
    inv = {v: k for k, v in lm.items()}
    class_names = [inv[i] for i in range(len(inv))]
else:
    n_classes = cm.shape[0]
    class_names = [str(i) for i in range(n_classes)]

def _annotated_confmat(M, title, outfile, fmt="d", normalize=False):
    """Save a confusion matrix heatmap (PNG, high DPI) with a white background."""
    fig_w = max(6, min(0.35 * M.shape[1], 20))
    fig_h = max(5, min(0.35 * M.shape[0], 20))
    fig = plt.figure(figsize=(fig_w, fig_h), dpi=600, facecolor="white")
    ax = fig.add_subplot(111)
    ax.set_facecolor("white")

    # Normalization if requested
    if normalize:
        with np.errstate(invalid="ignore", divide="ignore"):
            row_sums = M.sum(axis=1, keepdims=True)
            M_plot = np.divide(M, row_sums, out=np.zeros_like(M, dtype=float), where=row_sums!=0) * 100.0
    else:
        M_plot = M

    # visually clean colormap (e.g., viridis, cividis, or blues)
    im = ax.imshow(M_plot, interpolation="nearest", cmap="Blues")
    cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.ax.set_ylabel("%" if normalize else "Count", rotation=90, va="center", fontsize=8)

    ax.set_title(title, pad=12, fontsize=10)
    ax.set_xlabel("Predicted", fontsize=9)
    ax.set_ylabel("True", fontsize=9)

    tick_idx = np.arange(M.shape[0])
    max_ticks = 50
    if len(class_names) <= max_ticks:
        ax.set_xticks(tick_idx)
        ax.set_xticklabels(class_names, rotation=90, fontsize=7)
        ax.set_yticks(tick_idx)
        ax.set_yticklabels(class_names, fontsize=7)
    else:
        ax.set_xticks(tick_idx)
        ax.set_xticklabels(tick_idx, rotation=90, fontsize=7)
        ax.set_yticks(tick_idx)
        ax.set_yticklabels(tick_idx, fontsize=7)

    # Annotate cells if matrix is not too large
    if M.shape[0] <= 50:
        display = M_plot if normalize else M.astype(int)
        fmt_str = "{:.1f}" if normalize else "{:d}"
        thresh = np.nanmax(M_plot) * 0.5 if np.isfinite(np.nanmax(M_plot)) else 0.0
        for i in range(M.shape[0]):
            for j in range(M.shape[1]):
                val = display[i, j]
                color = "white" if (np.isfinite(M_plot[i, j]) and M_plot[i, j] > thresh) else "black"
                ax.text(j, i, fmt_str.format(val),
                        ha="center", va="center", color=color, fontsize=6)

    fig.tight_layout()
    fig.savefig(outfile, bbox_inches="tight", facecolor="white")
    plt.close(fig)

# Save both versions (white background)
_annotated_confmat(cm, "Confusion Matrix (Counts)",
                   "/content/drive/MyDrive/code_data25/Basu/save_results/confusion_matrix_counts.png",
                   fmt="d", normalize=False)
_annotated_confmat(cm, "Confusion Matrix (Row-Normalized %)",
                   "/content/drive/MyDrive/code_data25/Basu/save_results/confusion_matrix_normalized.png",
                   fmt=".1f", normalize=True)

print("Saved white-background PNGs: confusion_matrix_counts.png, confusion_matrix_normalized.png")


Loaded weights. Missing keys: [] | Unexpected keys: []
Saved: test_accuracy.txt, classification_report.txt, confusion_matrix.txt, predictions.csv, per_class_accuracy.csv


/tmp/ipython-input-362078712.py:54: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  per_class_acc = df.groupby("y_true").apply(lambda g: float((g.y_true == g.y_pred).mean()))


{
  "device": "cuda",
  "batch_size": 1,
  "runs": 200,
  "per_sample_time_s_mean": 0.0001530160018801689,
  "per_sample_time_s_median": 0.0001505279988050461,
  "per_sample_time_s_p90": 0.000158720001578331,
  "per_sample_time_s_p95": 0.00017107839807867998,
  "per_sample_time_s_min": 0.00014745600521564483,
  "throughput_samples_per_s_mean": 6535.2642057863195,
  "params_total": 8404510,
  "flops_per_sample": 16807936,
  "gflops_per_sample": 0.016807936
}
Saved /content/drive/MyDrive/code_data25/Basu/save_results/inference_profile.json


/tmp/ipython-input-362078712.py:158: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  per_class_acc = df_eval.groupby("y_true").apply(lambda g: float((g.y_true == g.y_pred).mean()))


Saved: /content/drive/MyDrive/code_data25/Basu/save_results/per_class_accuracy_bar.png, /content/drive/MyDrive/code_data25/Basu/save_results/roc_curves_micro_macro_onevsrest.png, /content/drive/MyDrive/code_data25/Basu/save_results/tsne_features.png
Saved white-background PNGs: confusion_matrix_counts.png, confusion_matrix_normalized.png
